In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


#####################################################################
# RUN THIS CODE CELL IF YOU ARE RESUMING THE NOTEBOOK AFTER A BREAK #
#####################################################################

# allocate 50% of GPU memory (if you like, feel free to change this)
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf 
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 1.0
# set_session(tf.Session(config=config))

# watch for any changes in the sample_models module, and reload it automatically
%load_ext autoreload
%autoreload 2
# import NN architectures for speech recognition
from sample_models import *
# import function for training acoustic model
from train_utils import train_model

/home/pemfir/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pemfir/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pemfir/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pemfir/

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
# specify the model
model_end = final_model(input_dim=161,units=512,recur_layers=3) 

# model_end = model_4

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, None, 161)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 512)         907264    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 512)         0         
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 512)         2048      
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 1024)        3148800   
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 1024)        4096      
___________________________

In [3]:
train_model(input_to_softmax=model_end, 
            pickle_path='model_end.pickle', 
            save_model_path='model_end.h5', 
            spectrogram=True) # change to False if you would like to use MFCC features

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
101/101 [==============================] - 124s 1s/step - loss: 238.3555 - val_loss: 211.6380
Epoch 2/20
101/101 [==============================] - 122s 1s/step - loss: 199.7134 - val_loss: 190.2719
Epoch 3/20
101/101 [==============================] - 123s 1s/step - loss: 179.3932 - val_loss: 169.9347
Epoch 4/20
101/101 [==============================] - 124s 1s/step - loss: 165.4828 - val_loss: 159.0617
Epoch 5/20
101/101 [==============================] - 123s 1s/step - loss: 154.4504 - val_loss: 149.3550
Epoch 6/20
101/101 [==============================] - 122s 1s/step - loss: 143.8905 - val_loss: 145.2875
Epoch 7/20
101/101 [==============================] - 123s 1s/step - loss: 134.8992 - val_loss: 140.8590
Epoch 8/20
101/101 [==============================] - 123s 1s/step - loss: 125.7273 - val_l

__Question 2:__ Describe your final model architecture and your reasoning at each step.  

__Answer:__

<a id='step3'></a>
## STEP 3: Obtain Predictions

We have written a function for you to decode the predictions of your acoustic model.  To use the function, please execute the code cell below.

In [4]:
import numpy as np
from data_generator import AudioGenerator
from keras import backend as K
from utils import int_sequence_to_text
from IPython.display import Audio

def get_predictions(index, partition, input_to_softmax, model_path):
    """ Print a model's decoded predictions
    Params:
        index (int): The example you would like to visualize
        partition (str): One of 'train' or 'validation'
        input_to_softmax (Model): The acoustic model
        model_path (str): Path to saved acoustic model's weights
    """
    # load the train and test data
    data_gen = AudioGenerator()
    data_gen.load_train_data()
    data_gen.load_validation_data()
    
    # obtain the true transcription and the audio features 
    if partition == 'validation':
        transcr = data_gen.valid_texts[index]
        audio_path = data_gen.valid_audio_paths[index]
        data_point = data_gen.normalize(data_gen.featurize(audio_path))
    elif partition == 'train':
        transcr = data_gen.train_texts[index]
        audio_path = data_gen.train_audio_paths[index]
        data_point = data_gen.normalize(data_gen.featurize(audio_path))
    else:
        raise Exception('Invalid partition!  Must be "train" or "validation"')
        
    # obtain and decode the acoustic model's predictions
    input_to_softmax.load_weights(model_path)
    prediction = input_to_softmax.predict(np.expand_dims(data_point, axis=0))
    output_length = [input_to_softmax.output_length(data_point.shape[0])] 
    pred_ints = (K.eval(K.ctc_decode(
                prediction, output_length)[0][0])+1).flatten().tolist()
    
    # play the audio file, and display the true and predicted transcriptions
    print('-'*80)
    Audio(audio_path)
    print('True transcription:\n' + '\n' + transcr)
    print('-'*80)
    print('Predicted transcription:\n' + '\n' + ''.join(int_sequence_to_text(pred_ints)))
    print('-'*80)

Use the code cell below to obtain the transcription predicted by your final model for the first example in the training dataset.

In [5]:
get_predictions(index=0, 
                partition='train',
                input_to_softmax=final_model(input_dim=161,units=200,recur_layers=2)  , 
                model_path='./results/model_end.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, None, 161)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 512)         907264    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 512)         0         
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 512)         2048      
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 400)         855600    
_________________________________________________________________
batch_normalization_4 (Batch (None, None, 400)         1600      
_________________________________________________________________
rnn_dropout0 (Dropout)       (None, None, 400)         0         
__________

ValueError: You are trying to load a weight file containing 9 layers into a model with 7 layers.

Use the next code cell to visualize the model's prediction for the first example in the validation dataset.

In [ ]:
get_predictions(index=0, 
                partition='validation',
                input_to_softmax=final_model(input_dim=161,units=200,recur_layers=2) , 
                model_path='./results/model_end.h5')

In [ ]:
907264/(512*161)